In [1]:
import numpy as np
from LightTools import LTAPIx
import time
import matplotlib.pyplot as plt
import re
PID = 14324
loc = LTAPIx.LTLocator()
lt = loc.GetLTAPIFromPID(PID)
lt.Message('Script LT connection established')

0

In [28]:
class Zone:
    def __init__(self, name, shape_type, shape_paras, opt_type, opt_paras) -> None:
        self.name = name
        self.shape_type = shape_type        
        self.shape_paras = shape_paras
        self.opt_type = opt_type
        self.opt_paras = opt_paras
        if shape_type == 'Polygonal':
            self.shape_paras_LT = {**{'X_Offset':0, 'Y_Offset':0, 'Theta_Rotation':0}, **self.array_to_dict(shape_paras, 'Vertex_X_At', 'Vertex_Y_At')}
        elif shape_type == 'Circle':
            self.shape_paras_LT = {'X_Offset':shape_paras[0], 'Y_Offset':shape_paras[1], 'Theta_Rotation':0, 'Radius':shape_paras[2]}
        else:
            raise NameError('Only Polygonal and Circle are allowed')
        return
    
    def check_tabulated(self, s):
        # Use regex to find a number inside square brackets
        match = re.search(r'([^\[]+)\[(\d+)\]', s)
        if match:
            text_before_bracket = match.group(1)  # Extract the text before the bracket
            number_in_bracket = int(match.group(2))  # Extract the number inside the bracket as an integer
            return text_before_bracket, number_in_bracket
        return s, None  # Return None if the pattern isn't found

    def array_to_dict(self, input_array, *labels):
        output_dict = {}
        label_list = list(labels)
        num_columns = input_array.shape[1]
        if len(label_list) != num_columns:
            raise ValueError(f"Number of labels ({len(label_list)}) must match the number of columns ({num_columns}) in the input array.")
        for i, row in enumerate(input_array, start=1):
            for j in range(num_columns):
                output_dict[f"{label_list[j]}[{i}]"] = row[j]
        return output_dict

    def set_zone(self, lt, surface_key, name, shape, paras):
        lt.Cmd("\O" + lt.Str(surface_key))
        lt.Cmd(lt.Str("Add Property Zone"))
        lt.Cmd("\Q")
        lt.DbSet(f'{surface_key}.ZONE[@Last]','Name', f'{name}')
        lt.DbSet(f'{surface_key}.ZONE[{name}].RECT_ZONE_EXTENT[RectangularZone]', 'Element Shape', shape) 
        shape_keys = {'Circle':'CIRC_ZONE_EXTENT[CircularZone]', 'Polygonal':'POLYGON_ZONE_EXTENT[PolygonalZone]'}
        lt.SetOption("DbUpdate", 0)
        for k_str in paras.keys():        
            k, ind = self.check_tabulated(k_str)
            if  ind is not None:
                # print(k, ind, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, None, ind))
                lt.DbSet(f'{surface_key}.ZONE[{name}].{shape_keys[shape]}', k, paras[k_str], ind)
            else:                
                # print(k, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k))
                lt.DbSet(f'{surface_key}.ZONE[{name}].{shape_keys[shape]}', k, paras[k_str])    
        
        lt.Cmd("\O" + lt.Str(f'{surface_key}.ZONE[{name}]'))
        lt.Cmd("PropertyName=" + f'"{name}"')
        lt.Cmd("\Q")        
        lt.SetOption("DbUpdate", 1)

    def add_zone_property(self, lt, name, opt_type, paras):
        lt.Cmd("\O" + lt.Str('PROPERTY_MANAGER[Optical Properties Manager]'))
        lt.Cmd("AddNew=""")
        lt.Cmd("\Q")
        lt.DbSet('PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[@Last]','Name', f'{name}')
        lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}]', 'Simple Type', opt_type)    
        lt.SetOption("DbUpdate", 0)
        if paras is not None:
            for f in paras.keys():
                for k_str in paras[f].keys():
                    # lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, paras[f][k])
                    k, ind = self.check_tabulated(k_str)
                    if  ind is not None:
                        # print(k, ind, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, None, ind))
                        lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, paras[f][k_str], ind)
                    else:                
                        # print(k, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k))
                        lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, paras[f][k_str])
        lt.SetOption("DbUpdate", 1)


    def to_LT(self, lt, surface_key):
        self.add_zone_property(lt, self.name, self.opt_type, self.opt_paras)
        self.set_zone(lt, surface_key, self.name, self.shape_type, self.shape_paras_LT)
        return
    


In [3]:
# Optical Property Parameters
paras_1 = {'RULED_DOE_DIRECTION_ZONE[RuledGrating]':{'Grating_Period':0.0005, 'Blaze_Type': 'Manual', 'Grating_Vector_X':0, 'Grating_Vector_Y':1, 'Grating_Vector_Z':0},
           'DIFFRACTION_EFFICIENCY_ZONE[diffEffTable]':{'Minimum_Order':0, 'Maximum_Order':1, 'EnabledTransmittedOrderAt[1]':0, 'ReflectedEfficiencyAt[1]':0.9, 'EnabledTransmittedOrderAt[2]':0, 
                                                        'ReflectedEfficiencyAt[2]':0.1, 'TransmittedEfficiencyAt[1]':0, 'TransmittedEfficiencyAt[2]':0}
           }

paras_2 = {'RULED_DOE_DIRECTION_ZONE[RuledGrating]':{'Grating_Period':0.0005, 'Blaze_Type': 'Manual', 'Grating_Vector_X':1, 'Grating_Vector_Y':0, 'Grating_Vector_Z':0},
           'DIFFRACTION_EFFICIENCY_ZONE[diffEffTable]':{'Minimum_Order':0, 'Maximum_Order':1, 'EnabledTransmittedOrderAt[1]':0, 'ReflectedEfficiencyAt[1]':0.8, 'EnabledTransmittedOrderAt[2]':0, 
                                                        'ReflectedEfficiencyAt[2]':0.2, 'TransmittedEfficiencyAt[1]':0, 'TransmittedEfficiencyAt[2]':0}
           }

In [29]:
surface_key = 'LENS_MANAGER[1].COMPONENTS[Components].SOLID[Cylinder_3].CYLINDER_PRIMITIVE[CylinderPrimitive_1].PLANAR_SURFACE[FrontSurface]'


epe_shape_pts = np.loadtxt('EPE.txt')
oc_shape_pts = np.loadtxt('oc.txt')


ic = Zone('IC', 'Circle', (-41.89725171, 29.00955127, 1.5), 'Linear Grating', None)
epe = Zone('EPE', 'Polygonal', epe_shape_pts, 'Measured BSDF', None)
oc = Zone('OC', 'Polygonal', oc_shape_pts, 'RSoft BSDF UDOP', None)



In [30]:
ic.to_LT(lt, surface_key)
epe.to_LT(lt, surface_key)
oc.to_LT(lt, surface_key)

In [14]:
name = 'IC'
print(f'PropertyName="{name}"')

PropertyName="IC"
